In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import joblib
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV

#torch stuff
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# check xgboost version
from xgboost import XGBRegressor


import holidays
us_holidays = holidays.country_holidays('US')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [4]:
df = pd.read_csv(f'{data_dir}/final_model_input_partial_scale_4.csv')

df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])
df = df.sort_values(by=['start_station_cluster', 'started_at_hourly'])

df_test = df[pd.to_datetime(
    df['started_at_hourly']) >= pd.to_datetime("2024-01-01 00:00:00")]
df_train = df[pd.to_datetime(
    df['started_at_hourly']) < pd.to_datetime("2024-01-01 00:00:00")]

del df

In [5]:
target_scaler = joblib.load(f'{models_dir}/target_scaler.sav')

In [6]:
execfile('/content/drive/MyDrive/Thesis/models_training/model_training.py')

Using device: cpu


# Weather Lag and Lag

In [7]:
FEATURES = ['start_station_cluster', 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            'is_holiday',
            #'flag_added',
            'temp',
            'dwpt',
            'rhum',
            'prcp',
            'wdir',
            'wspd',
            'pres',
            'coco',
            "demand_lag_1_h",
            "demand_lag_2_h",
            "demand_lag_24_h",
            "temp_lag_1_h",
            "temp_lag_2_h",
            "temp_lag_24_h",
            "prcp_lag_1_h",
            "prcp_lag_2_h",
            "prcp_lag_24_h",
            "rhum_lag_1_h",
            "rhum_lag_2_h",
            "rhum_lag_24_h",
            "wspd_lag_1_h",
            "wspd_lag_2_h",
            "wspd_lag_24_h",
            "coco_lag_1_h",
            "coco_lag_2_h",
            "coco_lag_24_h",
            "total_demand_1h",
            "demand_degrees_1h",
]

## XGBoost

In [8]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='xgb')

overall
MSE: 3.680589537470945
RMSE: 1.9184862619969278
MAE: 1.0251868492937535
MAPE: 0.41571022142206315

Non-zero
MSE: 7.468849180206185
RMSE: 2.732919534162355
MAE: 1.7112843503707298
MAPE: 0.35639052922218767

Zeros
MSE: 0.5831888692532394
RMSE: 0.7636680360295561
MAE: 0.4642118743144978
MAPE: 0.4642118743144978


In [ ]:
#11-01
3.1819311789251117
1.783796843512487
0.374929281500934

#11-09 scaled
3.668979611469117
1.9154580683139781
0.4155907549079753

#11-09 unscaled
3.646843601419425
1.9096710715249956
0.41661259127538636

#11-28
3.694719359705295
1.9221652789771475
0.41533332978086984

#01-11
# overall
# MSE: 3.680589537470945
# RMSE: 1.9184862619969278
# MAE: 1.0251868492937535
# MAPE: 0.41571022142206315
# Non-zero
# MSE: 7.468849180206185
# RMSE: 2.732919534162355
# MAE: 1.7112843503707298
# MAPE: 0.35639052922218767
# Zeros
# MSE: 0.5831888692532394
# RMSE: 0.7636680360295561
# MAE: 0.4642118743144978
# MAPE: 0.4642118743144978

0.41533332978086984

In [ ]:
suf = "_d_xgb_01_11"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## LightGBM

In [9]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='lgbm')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.168412 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3120
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 37
[LightGBM] [Info] Start training from score 1.005143
overall
MSE: 3.817927494977813
RMSE: 1.9539517637285249
MAE: 1.0565380543067031
MAPE: 0.4309508302291346

Non-zero
MSE: 7.723996836863059
RMSE: 2.779207951352878
MAE: 1.7573510894769677
MAPE: 0.36664269192606547

Zeros
MSE: 0.6242018978261752
RMSE: 0.7900644896628218
MAE: 0.48353119412624673
MAPE: 0.48353119412624673


In [ ]:
#11_15"
3.8287248753558356
1.9567127728299407
0.4312900588663665

#11-28
3.8084135604311347
1.9515157084766535
0.4313976119662708

#01-11
# overall
# MSE: 3.817927494977813
# RMSE: 1.9539517637285249
# MAE: 1.0565380543067031
# MAPE: 0.4309508302291346
# Non-zero
# MSE: 7.723996836863059
# RMSE: 2.779207951352878
# MAE: 1.7573510894769677
# MAPE: 0.36664269192606547
# Zeros
# MSE: 0.6242018978261752
# RMSE: 0.7900644896628218
# MAE: 0.48353119412624673
# MAPE: 0.48353119412624673

0.4313976119662708

In [ ]:
suf = "_d_lgbm_01_11"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model


## Random Forest

In [10]:
model, model_preds = train_model(df_train, df_test, FEATURES, model='rf')

overall
MSE: 4.140577401870537
RMSE: 2.0348408787594514
MAE: 1.090460606490045
MAPE: 0.45904893058533747

Non-zero
MSE: 8.247253237149373
RMSE: 2.871803133424952
MAE: 1.7813517195756792
MAPE: 0.3776953049371889

Zeros
MSE: 0.7828296031579008
RMSE: 0.8847765837531534
MAE: 0.5255662230121492
MAPE: 0.5255662230121492


In [ ]:
3.6314578551418113
1.905638437674317
0.3965058032591828

#11-01
3.594735773915557
1.8959788432141211
0.4107832155834339

#11-28
4.14255731366025
2.0353273234691884
0.4592568013827876

#01-11
# overall
# MSE: 4.140577401870537
# RMSE: 2.0348408787594514
# MAE: 1.090460606490045
# MAPE: 0.45904893058533747
# Non-zero
# MSE: 8.247253237149373
# RMSE: 2.871803133424952
# MAE: 1.7813517195756792
# MAPE: 0.3776953049371889
# Zeros
# MSE: 0.7828296031579008
# RMSE: 0.8847765837531534
# MAE: 0.5255662230121492
# MAPE: 0.5255662230121492

0.4592568013827876

In [ ]:
suf = "_d_rf_01_11"

model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model